In [150]:
import numpy as np
import sympy

In [151]:
perch_length = np.array(
    [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 
     36.5, 36.0, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
     )
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [152]:
class Rnn:
  """
  순환 신경망의 계산
  """
  # 입력 가중치
  input_w = []

  # 이전 출력에 대한 가중치
  before_w = []

  # 출력 가중치
  result_w = []

  # 편향 값 bais
  bias = []

  def cal_rnn(self, input, output, input_w = [], bias = []):
    """
    순환 신경망 은닉층까지의 계산
    args
      input : 입력 데이터
      output : 원하는 출력 형태
    return 
      순환 신경망의 계산 결과
    """
    # 이전 출력에 대한 입력, 첫 부분의 경우 이 값이 0이다.
    before = np.zeros((output.shape[0], output.shape[1]))

    # 이전 출력에 대한 가중치
    before_w = np.random.rand(output.shape[0], output.shape[0])

    self.before_w.append(before_w)

    # 가중치 w의 생성, 
    if not input_w:
      input_w = np.random.rand(output.shape[0], input.shape[0])
    
    if not bias:
      bias = np.random.rand()

    self.input_w.append(input_w)

    self.bias.append(bias)

    return (before_w @ before) + (input_w @ input) + bias

  def cal_result(self, input, output, w = []):
    """
    순환 신경망의 은닉층 출력에서 최종 결과값으로의 계산
    args
      input : 입력 데이터
      output : 원하는 출력 형태
    return 
      행렬 곱 연산 결과
    """
    # 가중치 생성
    if not w:
      result_w = np.random.rand(output.shape[0], output.shape[0])

    self.result_w.append(result_w)

    return result_w @ input

In [153]:
class Activation:
  """
  데이터의 활성화 함수 연산 결과를 얻기 위함
  추가로 역전파 계산에 필요한 활성화 함수의 미분 함수
  """
  # 활성화 함수
  fx = 0
  # 활성화 함수의 미분 함수
  d_fx = 0

  def sigmoid_fun(self, input):
    """
    시그모이드 함수
    자연 상수의 지수 함수를 사용한 0~1 사이의 값 출력
    Args:
      input : 입력값
    Return:
      0~1 사이의 값
    """
    x = sympy.Symbol('x')
    result = []

    self.fx = 1/(1+np.e**(-x))

    self.d_fx = sympy.diff(self.fx, x)

    for i in range(input.shape[0]):
      result.append(self.fx.subs([(x, input[i][0])]))

    return np.array(result).reshape(-1, 1)

  def d_sigmoid_fun(self, input):
    """
    시그모이드 함수의 미분값, 노드의 출력이 입력된다.
    arg
      input : 입력값
    return
      시그모이드 미분 함수의 계산 값
    """
    result = []

    x = sympy.Symbol('x')

    for i in range(input.shape[0]):
      result.append(self.d_fx.subs([(x,input[i][0])]))

    return np.array(result).reshape(-1, 1)

In [154]:
class Error:
  """
  오차 함수와 오차 함수의 미분 함수를 얻을 수 있다.  
  """
  # 오차 함수
  fx = 0
  
  # 오차 함수의 미분값
  d_fx = 0
  
  def RMSE(self, target, predict):
    """
    일반적인 제곱 오차 함수
    args:
      target : 목표 변수
      predict : 예측 변수
    return:
      제곱 오차 평균 값
    """
    result = (predict - target)**2
    self.result = np.mean(result)

    t = sympy.Symbol('t')
    y = sympy.Symbol('y')

    self.fx = 1/2*((y-t)**2)

    self.d_fx = sympy.diff(self.fx, y)

    return self.result

  def d_RMSE(self, target, predict):
    """
    RMSE 의 미분 함수
    args
      target : 목표 변수
      predict : 예측 변수
    return:
      미분 함수의 계산 값
    """
    result = []

    t = sympy.Symbol('t')
    y = sympy.Symbol('y')

    for i in range(target.shape[0]):
      result.append(self.d_fx.subs([(t, target[i][0]), (y, predict[i][0])]))
    
    return np.array(result).reshape(-1, 1)


In [155]:
input = perch_length[:3].reshape(-1,1)

In [156]:
target = perch_length[:4].reshape(-1,1)

In [157]:
output = np.zeros((4,1))

In [158]:
input.shape

(3, 1)

In [159]:
output.shape

(4, 1)

In [160]:
rnn = Rnn()

In [161]:
rnn_hidden_input = rnn.cal_rnn(input, output)
rnn_hidden_input

array([[14.61555032],
       [27.62500626],
       [ 7.10682331],
       [24.86375399]])

In [162]:
act = Activation()

In [163]:
rnn_hidden_output = act.sigmoid_fun(rnn_hidden_input)
rnn_hidden_output

array([[0.999999550689106],
       [0.999999999998994],
       [0.999181176879580],
       [0.999999999984085]], dtype=object)

In [164]:
rnn_result = rnn.cal_result(rnn_hidden_output, output)
rnn_result

array([[1.72907351616934],
       [1.38385511903087],
       [2.12730161245530],
       [1.94484919382546]], dtype=object)

델타 연산

예측 노드의 델타,

오차 함수의 미분값과 동일하다.(활성화 함수로 항등 함수를 사용했기 때문)

In [165]:
err = Error()

In [166]:
result = err.RMSE(target, rnn_result)
result

141.276093291292

In [167]:
result_delta = err.d_RMSE(target, rnn_result)
result_delta

array([[-6.67092648383066],
       [-12.3161448809691],
       [-12.8726983875447],
       [-14.2551508061745]], dtype=object)

예측 노드의 델타를 사용한 result_w 값 업데이트

In [168]:
rnn_hidden_output

array([[0.999999550689106],
       [0.999999999998994],
       [0.999181176879580],
       [0.999999999984085]], dtype=object)

In [169]:
d_result_w = result_delta * rnn_hidden_output
d_result_w

array([[-6.67092348651071],
       [-12.3161448809567],
       [-12.8621579244828],
       [-14.2551508059477]], dtype=object)

예측 노드의 델타를 이용한 은닉층 노드의 delta 값 계산

In [170]:
result_delta

array([[-6.67092648383066],
       [-12.3161448809691],
       [-12.8726983875447],
       [-14.2551508061745]], dtype=object)

In [171]:
rnn.result_w[0]

array([[0.0037174 , 0.3119566 , 0.98930572, 0.42490386],
       [0.26105727, 0.85425761, 0.20917303, 0.0595386 ],
       [0.50509885, 0.71417804, 0.85119491, 0.05752702],
       [0.7930414 , 0.72072046, 0.14886736, 0.28234222]])

은닉층의 각 노드들의 변화에 따른 오차 함수의 변화량

In [172]:
d_hidden_c = rnn.result_w[0] @ result_delta
d_hidden_c

array([[-22.6590039717964],
       [-15.8040074143507],
       [-23.9426291627106],
       [-20.1079741286942]], dtype=object)

위 값을 통한 은닉층 각 노드들의 델타 계산, 활성화 함수의 미분값과의 계산

In [173]:
d_hidden_c

array([[-22.6590039717964],
       [-15.8040074143507],
       [-23.9426291627106],
       [-20.1079741286942]], dtype=object)

In [174]:
act.d_sigmoid_fun(rnn_hidden_output)

array([[0.196611974064854],
       [0.196611933241573],
       [0.196686317812250],
       [0.196611933242928]], dtype=object)

In [175]:
hidden_delta = d_hidden_c * act.d_sigmoid_fun(rnn_hidden_output)
hidden_delta

array([[-4.45503150123827],
       [-3.10725645069964],
       [-4.70918756875775],
       [-3.95346766704134]], dtype=object)

은닉층의 델타값을 통한 input_w 의 계산

In [176]:
hidden_delta

array([[-4.45503150123827],
       [-3.10725645069964],
       [-4.70918756875775],
       [-3.95346766704134]], dtype=object)

In [177]:
rnn.input_w[0].shape

(4, 3)

In [179]:
d_input_w = (hidden_delta.T @ rnn.input_w[0]).T
d_input_w

array([[-9.58717866136145],
       [-8.33815413826323],
       [-5.64186208114704]], dtype=object)